In [1]:
# dependencies

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import datetime
import plotly.express as px

from sqlalchemy import create_engine

pd.set_option('display.max_columns', None)

In [2]:
# database connection

db_password = "UnitCircle42!"
db_user = "postgres"
db_name = "dot"
endpoint = "awakedb.cre3f7yk1unp.us-west-1.rds.amazonaws.com"

connection_string = f"postgresql://{db_user}:{db_password}@{endpoint}:5432/{db_name}"
engine = create_engine(connection_string)

In [128]:
def get_data_from_csv():
    df = pd.read_sql("""
            SELECT * 
            FROM level_2
            WHERE year > '2020';
            """
            ,con = engine)
    return df
df = get_data_from_csv()

### MASTER DATA ###
all_sales = df.copy()
all_sales = all_sales.convert_dtypes()

all_sales.head()

,date,sale_origin,market_segment,parent_customer,customer,item,qty,usd,cad,month,year
0,2024-01-17,unl,Online Distributor,Amazon,Amazon SC,50ct Pouch - Dark Choc Bites,1.0,34.9875,46.65,January,2024
1,2024-01-17,unl,Online Distributor,Amazon,Amazon SC,50ct Pouch - Mint Choc Bites,1.0,34.9875,46.65,January,2024
2,2024-01-17,unl,Online Distributor,Amazon,Amazon SC,50ct Pouch - Dark Choc Bites,1.0,34.9875,46.65,January,2024
3,2024-01-17,dot,Alternate Retail,Vistar Retail,VISTAR-RETAIL CENTRAL,AWAKE CHOCOLATE AWAKE CAFF MILK CHOC- 6X12PK M...,13.0,1112.02,1478.99,January,2024
4,2024-01-17,unl,Online Distributor,Amazon,Amazon SC,50ct Pouch - Dark Choc Bites,1.0,34.9875,46.65,January,2024


In [ ]:
segment = st.sidebar.multiselect(
    label='Market Segment',
    options=all_sales['market_segment'].unique().to_list(),
    default=all_sales['market_segment'].unique().to_list(),
)

In [132]:
all_sales.columns

Index(['date', 'sale_origin', 'market_segment', 'parent_customer', 'customer',
       'item', 'qty', 'usd', 'cad', 'month', 'year'],
      dtype='object')

In [130]:
all_sales['date'] = pd.to_datetime(all_sales['date'])
all_sales['date'] = all_sales['date'].dt.normalize()
all_sales['date'] = all_sales['date'].dt.floor('D')
all_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144287 entries, 0 to 144286
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   date             144287 non-null  datetime64[ns]
 1   sale_origin      144287 non-null  string        
 2   market_segment   144287 non-null  string        
 3   parent_customer  144287 non-null  string        
 4   customer         144287 non-null  string        
 5   item             144287 non-null  string        
 6   qty              144287 non-null  Float64       
 7   usd              144287 non-null  Float64       
 8   cad              144287 non-null  Float64       
 9   month            144287 non-null  string        
 10  year             144287 non-null  Int64         
dtypes: Float64(3), Int64(1), datetime64[ns](1), string(6)
memory usage: 12.7 MB


In [84]:
samples_23 = int(all_sales[(all_sales.date.dt.year==2023) & (all_sales.market_segment=='Samples')]['qty'].sum())
samples_22 = int(all_sales[(all_sales.date.dt.year==2022) & (all_sales.market_segment=='Samples')]['qty'].sum())
yoy_samples = int(samples_23-samples_22)
yoy_samples_perc = round(int(samples_23-samples_22)/samples_22,2)

In [90]:
yoy_samples_perc

-0.4

In [37]:
online_23 = all_sales[(all_sales['year'] == 2023) & (all_sales['market_segment'] == 'Online Direct') | (all_sales['market_segment'] == 'Online Distributor')].usd.sum()
# online_22 = all_sales[(all_sales['year'] == 2022) & (all_sales['market_segment'] == 'Online Direct') | (all_sales['market_segment'] == 'Online Distributor')].usd.sum()
# yoy_online = int(online_23-online_22)
# yoy_online_perc = round(int(online_23-online_22) / online_22,2)

In [74]:
online = all_sales[(all_sales.market_segment == 'Online Direct') | (all_sales.market_segment=='Online Distributor')]
online_23 = int(online[online.date.dt.year==2023].usd.sum())

2260971

In [58]:
all_sales[(all_sales['market_segment'] == 'Online Direct') | (all_sales['market_segment'] == 'Online Distributor')].groupby(['market_segment','year'],as_index=False)['usd'].sum()

,market_segment,year,usd
0,Online Direct,2018,265651.3425
1,Online Direct,2019,315065.67
2,Online Direct,2020,510627.6
3,Online Direct,2021,284134.4025
4,Online Direct,2022,67980.2175
5,Online Direct,2023,50653.665
6,Online Direct,2024,955.3725
7,Online Distributor,2018,489535.9951
8,Online Distributor,2019,654521.051
9,Online Distributor,2020,742274.48


In [37]:


# invoice date cleanup
all_sales['date'] = pd.to_datetime(all_sales['date'])
all_sales['date'] = all_sales['date'].dt.normalize()
all_sales['date'] = all_sales['date'].dt.floor('D')
all_sales.sort_values(by='usd',ascending=False,inplace=True)

sales_23 = int(all_sales[all_sales['date'].dt.year == 2023].usd.sum())
sales_23

9614740